In [42]:
# importing the necessary libraries
import pandas as pd
import zipfile


In [48]:
#Reading the file
file_path = r'C:\Users\Julie Musangi\Desktop\DS Course Material\Phase-5\consumer_complaints_analysis\data_comp.zip'

with zipfile.ZipFile(file_path) as z:
    print(z.namelist())  # To list the files inside the zip file
    data = pd.read_csv(z.open('consumer_complaints.csv'), index_col=0)

['consumer_complaints.csv']


In [49]:
data.head()

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...
